In [ ]:
import os
from pathlib import Path

import cv2


def read_yolo_labels(label_path):
    """Read YOLO format labels from txt file"""
    labels = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                line = line.strip()
                if line:
                    parts = line.split()
                    class_id = int(parts[0])
                    x_center = float(parts[1])
                    y_center = float(parts[2])
                    width = float(parts[3])
                    height = float(parts[4])
                    labels.append([class_id, x_center, y_center, width, height])
    return labels


def write_yolo_labels(labels, output_path):
    """Write YOLO format labels to txt file"""
    with open(output_path, "w") as f:
        for label in labels:
            class_id, x_center, y_center, width, height = label
            f.write(
                f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"
            )


def transform_labels(labels, transform_type):
    """Transform YOLO labels based on transformation type"""
    transformed_labels = []

    for label in labels:
        class_id, x_center, y_center, width, height = label

        if transform_type == "rot90":  # 90° clockwise
            new_x = 1 - y_center
            new_y = x_center
            new_width = height
            new_height = width

        elif transform_type == "rot180":  # 180° rotation
            new_x = 1 - x_center
            new_y = 1 - y_center
            new_width = width
            new_height = height

        elif transform_type == "rot270":  # 270° clockwise
            new_x = y_center
            new_y = 1 - x_center
            new_width = height
            new_height = width

        elif transform_type == "flip_h":  # Horizontal flip
            new_x = 1 - x_center
            new_y = y_center
            new_width = width
            new_height = height

        elif transform_type == "flip_v":  # Vertical flip
            new_x = x_center
            new_y = 1 - y_center
            new_width = width
            new_height = height

        elif transform_type == "flip_h_rot90":  # Horizontal flip + 90° rotation
            # First horizontal flip
            temp_x = 1 - x_center
            temp_y = y_center
            # Then 90° rotation
            new_x = 1 - temp_y
            new_y = temp_x
            new_width = height
            new_height = width

        elif transform_type == "flip_v_rot90":  # Vertical flip + 90° rotation
            # First vertical flip
            temp_x = x_center
            temp_y = 1 - y_center
            # Then 90° rotation
            new_x = 1 - temp_y
            new_y = temp_x
            new_width = height
            new_height = width

        else:
            raise ValueError(f"Unknown transform type: {transform_type}")

        transformed_labels.append([class_id, new_x, new_y, new_width, new_height])

    return transformed_labels


def apply_image_transform(image, transform_type):
    """Apply transformation to image"""
    if transform_type == "rot90":
        return cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    elif transform_type == "rot180":
        return cv2.rotate(image, cv2.ROTATE_180)
    elif transform_type == "rot270":
        return cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    elif transform_type == "flip_h":
        return cv2.flip(image, 1)  # Horizontal flip
    elif transform_type == "flip_v":
        return cv2.flip(image, 0)  # Vertical flip
    elif transform_type == "flip_h_rot90":
        # First horizontal flip, then 90° rotation
        flipped = cv2.flip(image, 1)
        return cv2.rotate(flipped, cv2.ROTATE_90_CLOCKWISE)
    elif transform_type == "flip_v_rot90":
        # First vertical flip, then 90° rotation
        flipped = cv2.flip(image, 0)
        return cv2.rotate(flipped, cv2.ROTATE_90_CLOCKWISE)
    else:
        raise ValueError(f"Unknown transform type: {transform_type}")


def generate_augmented_images(image_path, output_dir=None):
    """
    Generate 7 augmented versions of an image and its labels

    Args:
        image_path: Path to the input image
        output_dir: Directory to save augmented images (optional)
    """
    image_path = Path(image_path)

    # Determine paths
    if output_dir is None:
        output_dir = image_path.parent.parent / "augmented"
    else:
        output_dir = Path(output_dir)

    # Create output directories
    output_images_dir = output_dir / "images"
    output_labels_dir = output_dir / "labels"
    output_images_dir.mkdir(parents=True, exist_ok=True)
    output_labels_dir.mkdir(parents=True, exist_ok=True)

    # Read original image
    image = cv2.imread(str(image_path))
    if image is None:
        raise ValueError(f"Could not read image: {image_path}")

    # Determine label path
    image_name = image_path.stem
    labels_dir = image_path.parent.parent / "labels"
    label_path = labels_dir / f"{image_name}.txt"

    # Read original labels
    original_labels = read_yolo_labels(label_path)

    # Define transformations
    transformations = {
        "rot90": "90deg",
        "rot180": "180deg",
        "rot270": "270deg",
        "flip_h": "flip_horizontal",
        "flip_v": "flip_vertical",
        "flip_h_rot90": "flip_h_rot90",
        "flip_v_rot90": "flip_v_rot90",
    }

    print(f"Processing image: {image_path.name}")
    print(f"Original labels count: {len(original_labels)}")

    # Generate augmented versions
    for transform_type, suffix in transformations.items():
        # Transform image
        transformed_image = apply_image_transform(image, transform_type)

        # Transform labels
        transformed_labels = transform_labels(original_labels, transform_type)

        # Save transformed image
        output_image_path = (
            output_images_dir / f"{image_name}_transform_{suffix}{image_path.suffix}"
        )
        cv2.imwrite(str(output_image_path), transformed_image)

        # Save transformed labels
        output_label_path = output_labels_dir / f"{image_name}_transform_{suffix}.txt"
        write_yolo_labels(transformed_labels, output_label_path)

        print(f"Generated: {output_image_path.name}")

    print(f"Augmentation complete! Files saved to: {output_dir}")

In [ ]:
from _models import get_all_images_pathlib


imgs: list[str] = get_all_images_pathlib("./train")

In [ ]:
for img in imgs:
    generate_augmented_images(img, os.path.dirname(os.path.dirname(img)))